## Process Outline
We need a set of manually categorized FERC plants with which to test the sklearn Classifier that we're making. We'll also use them to test the regression analysis that Alana is working on. What does this dataset look like, how do we create it?
* Pull FERC Plants table.
* Using a variety of search methods, to ensure that we get a wide variety of plants, identify sets of records in the FERC Form 1 Plants table that are comparable inter-year records.
* Searching can be done, for example, by matching a pattern against the plant name (which ought to show some cases in which the same plant is reported by different respondents, or where the ownership changes from year to year...), or by selecting all of the records from a given respondent, and identifying the records that pertain to a given plant consistently across years, even if the name changes.
* Diversity of plants is key -- we need to show a wide variety of different kinds of time series to the classifier so it knows how to deal with them.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import numpy as np
import pandas as pd
import sqlalchemy as sa
sys.path.append(os.path.abspath(os.path.join('..','..','..')))
from pudl import init, mcoe, analysis, settings, outputs
import pudl.extract.ferc1
import pudl.transform.ferc1
import pudl.constants as pc

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [3]:
# For some reason these things don't stick if they're in the same cell as the
# %matplotlib inline call above, but if they're separate, they work fine.
plt.style.use('ggplot')
mpl.rcParams['figure.figsize'] = (10,6)
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 56

In [4]:
ferc1_engine = pudl.extract.ferc1.connect_db()
ferc1_steam_raw = pd.read_sql('SELECT * FROM f1_steam;', ferc1_engine)
fc = pudl.transform.ferc1.FERCPlantClassifier()
fc._prepare_plants(ferc1_steam_raw)
ferc1_prep = fc._ferc1_steam

In [5]:
masks = [
    ((ferc1_prep.plant_name.str.match('.*cayuga.*')) &
     (ferc1_prep.plant_kind_cpi=='steam') &
     (ferc1_prep.yr_const=='1970')),

    ((ferc1_prep.plant_name.str.match('.*comanche.*')) &
     (ferc1_prep.respondent_id==148)),
]

In [6]:
# In this cell, you can mess around with various selections from the ferc plant
# dataframe, until it's just getting the plants you want to keep, and then add
# your new mask to the list of masks above.
#new_mask = (
#    (ferc1_prep.respondent_id==210) &
#    (ferc1_prep.yr_const=='2003')
#)
#ferc1_prep[new_mask]

In [7]:
ferc1_train = pd.DataFrame({}, columns=range(2004,2017))
new_plants = [ferc1_train]
for mask in masks:
    new_plants = new_plants + [ferc1_prep[mask][['record_id','report_year']].\
                                 set_index('report_year').transpose()]
ferc1_train = pd.concat(new_plants).reset_index().drop('index', axis=1)
ferc1_train

,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,2004_144_0_5,2005_144_0_5,2006_144_0_5,2007_144_0_5,2008_144_0_5,2009_144_0_5,2010_144_0_5,2011_144_1_4,2012_144_1_4,2013_144_1_4,2014_144_1_4,2015_144_1_4,2016_144_1_4
1,2004_148_0_5,2005_148_0_5,2006_148_0_5,2007_148_0_5,2008_148_0_5,2009_148_0_5,2010_148_0_5,2011_148_0_5,2012_148_0_5,2013_148_0_5,2014_148_0_5,2015_148_0_5,2016_148_0_5


In [10]:
ferc1_train.to_csv('ferc1_plantid_training.csv', index=False)

## FERC Masks from CES work
These masks won't work directly because of different dataframe names and available columns and column names... but they should give you hints on how to go grab these same plants.

In [9]:
ferc_masks['rodemacher'] = (ferc_df.plant_name.str.match('.*odemach.*2'))

ferc_masks['oklaunion_psok'] = ((ferc_df.plant_name.str.match('.*klaunion.*')) &
                                (ferc_df.utility_id==148))
ferc_masks['oklaunion_aeptx_north'] = ((ferc_df.plant_name.str.match('.*klaunion.*')) &
                                       (ferc_df.utility_id==189))
ferc_masks['oklaunion_aeptx_central'] = ((ferc_df.plant_name.str.match('.*klaunion.*')) &
                                         (ferc_df.utility_id==24))

ferc_masks['jeffrey_westar'] = ((ferc_df.plant_name.str.match('.*effrey.*')) &
                                (ferc_df.utility_id==191) &
                                (ferc_df.expns_fuel > 0.0))
ferc_masks['jeffrey_ksgeco'] = ((ferc_df.plant_name.str.match('.*effrey.*')) &
                                (ferc_df.utility_id==80) &
                                (ferc_df.expns_fuel > 0.0))
ferc_masks['jeffrey_kcpl'] = ((ferc_df.plant_name.str.match('.*effrey.*')) &
                              (ferc_df.utility_id==182) &
                              (ferc_df.expns_fuel > 0.0))

ferc_masks['lacygne_1_kcpl'] = (
    (ferc_df.plant_name.str.match('(La Cygne 50%|Lacygne 50%|Lacygne 1.*)')) &
    (ferc_df.utility_id==79)
)
ferc_masks['lacygne_2_kcpl'] = ((ferc_df.plant_name.str.match('.*ygne.*2.*')) &
                               (ferc_df.utility_id==79))
ferc_masks['lacygne_1_kge'] = ((ferc_df.plant_name.str.match('.*ygne.*#1.*')) &
                              (ferc_df.utility_id==80))
ferc_masks['lacygne_2_kge'] = ((ferc_df.plant_name.str.match('.*ygne.*#2.*')) &
                              (ferc_df.utility_id==80))

ferc_masks['sibley'] = ((ferc_df.utility_id==182) &
                        (ferc_df.plant_name=='Sibley'))

ferc_masks['george_neal_1'] = ((ferc_df.plant_name.str.match('.*Neal.*1')))
ferc_masks['george_neal_2'] = ((ferc_df.plant_name.str.match('.*Neal.*2')))
ferc_masks['george_neal_3_ipl'] = (
    (ferc_df.plant_name.str.match('.*Neal.*3') &
    (ferc_df.utility_id==281)))
ferc_masks['george_neal_3_midam'] = (
    (ferc_df.plant_name.str.match('.*Neal.*3') &
    (ferc_df.utility_id==210)))

ferc_masks['san_juan_psnm'] = ((ferc_df.plant_name.str.match('.*[Ss]an [Jj]uan.*')) &
                               (ferc_df.utility_id==147))
ferc_masks['san_juan_tepco'] = ((ferc_df.plant_name.str.match('.*[Ss]an [Jj]uan.*')) &
                                (ferc_df.utility_id==176))

ferc_masks['yorktown'] = (ferc_df.plant_name.str.match('.*orktown.*'))

ferc_masks['chesterfield'] = (ferc_df.plant_name.str.match('Chesterfield$'))

ferc_masks['rockport_aeg'] = ((ferc_df.plant_name.str.match('Rockport Total Aeg')) &
                              (ferc_df.utility_id==1))
ferc_masks['rockport_inmi'] = ((ferc_df.plant_name.str.match('Rockport Total I&M')) &
                               (ferc_df.utility_id==73))

ferc_masks['mt_storm'] = ((ferc_df.plant_name.str.match('Mount Storm$')) &
                          (ferc_df.utility_id==186))

###########################################################################
# Mitchell (West Virginia) plant. Originally owned by Ohio Power Company,
# then split between Kentucky Power Company, and AEP, subsequently Wheeling
ferc_masks['mitchell_opc'] = ((ferc_df.plant_name.str.match('.*[Mm]itchell.*')) &
                              (ferc_df.utility_id==127))
ferc_masks['mitchell_kpc'] = ((ferc_df.plant_name.str.match('.*Mitchell.*Share$')) &
                              (ferc_df.utility_id==81))
ferc_masks['mitchell_aep'] = ((ferc_df.plant_name.str.match('.*Mitchell.*Share$')) &
                              (ferc_df.utility_id==452))
ferc_masks['mitchell_wpc'] = ((ferc_df.plant_name.str.match('.*Mitchell.*Share$')) &
                              (ferc_df.utility_id==192))

ferc_masks['virginia_city'] = ((ferc_df.plant_name.str.match('.*irginia [Cc]ity.*')))

###########################################################################
# East Bend -- Ownership of Duke's share seems to have moved from one
# subsidiary to another. Also, why is only generator_id=2 showing up in EIA?
# Seems like another case of some bad NA values.
ferc_masks['east_bend_dukeoh'] = ((ferc_df.plant_name.str.match('.*East Bend.*')) &
                                  (ferc_df.utility_id==27))
ferc_masks['east_bend_daypl'] = ((ferc_df.plant_name.str.match('.*East Bend.*')) &
                                 (ferc_df.utility_id==42))
ferc_masks['east_bend_dukeky'] = ((ferc_df.plant_name.str.match('.*East Bend.*')) &
                                  (ferc_df.utility_id==178))

###########################################################################
# Harrison plant, owned by FirstEnergy, sold to subsidiary Mon Energy in
# 2013 -- at that point FERC reported capacity jumps to 2000MW. Only a 
# couple of complete years of cost data in EIA -- suffers from NA values.
# Last 2 years of FERC data are totally wacked -- neeeds to be cleaned up.
# Also, capacity reported doesn't really match up between EIA & FERC... we're
# missing a third unit, for which there's *no* whole year of data that's valid
# in EIA.  Gah.
ferc_masks['harrison'] = ((ferc_df.plant_name.str.match('.*arrison.*')) &
                                  (ferc_df.utility_id==101))

###########################################################################
# John Amos plant has 3 units. Jointly owned by Appalachian Power Co & 
# Ohio Power Co until 2013, after which point it is entirely App Pwr Co.
ferc_masks['amos_apco'] = ((ferc_df.plant_name.str.match('^Amos-Apco Share$')) &
                           (ferc_df.utility_id==6))
ferc_masks['amos_opco'] = ((ferc_df.plant_name.str.match('^Amos-Opco Share$')) &
                           (ferc_df.utility_id==127))
ferc_masks['amos_consolidated'] = ((ferc_df.plant_name.str.match('^Amos$')) &
                                   (ferc_df.utility_id==6))

###########################################################################
# Killen is slated for retirement mid-2018.  There's a weird dangling
# 12-18 MW generator of some kind being reported to FERC, which is why the
# DPL filter includes the nameplate capacity minimum of 100.0 MW. DPL also
# has fuel costs which are off by a factor of 1000x in 2015 only.
ferc_masks['killen_duke'] = ((ferc_df.plant_name.str.match('.*Killen.*')) &
                             (ferc_df.utility_id==27))
ferc_masks['killen_dpl'] = ((ferc_df.plant_name.str.match('.*Killen.*')) &
                              (ferc_df.utility_id==42) &
                              (ferc_df.nameplate_capacity_mw > 100.0))

###########################################################################
# J.M. Stuart plant has 4 coal fired units, and an ownership structure that
# has changed over the years. 2015 fuel cost numbers reported by DPL are off
# by a factor of 1000x.
ferc_masks['jm_stuart_duke'] = ((ferc_df.plant_name.str.match('.*Stuart.*')) &
                                (ferc_df.utility_id==27))
ferc_masks['jm_stuart_csp'] = ((ferc_df.plant_name.str.match('.*Stuart.*')) &
                               (ferc_df.utility_id==31))
ferc_masks['jm_stuart_opco'] = ((ferc_df.plant_name.str.match('.*Stuart.*')) &
                                (ferc_df.utility_id==127))
ferc_masks['jm_stuart_aep'] = ((ferc_df.plant_name.str.match('.*Stuart.*')) &
                               (ferc_df.utility_id==452))
ferc_masks['jm_stuart_dpl'] = ((ferc_df.plant_name.str.match('.*Stuart.*')) &
                               (ferc_df.utility_id==42) &
                               (ferc_df.nameplate_capacity_mw > 100.0))

ferc_masks['comanche'] = ((ferc_df.plant_name.str.match('Comanche')) &
                          (ferc_df.utility_id==145))

ferc_masks['pawnee'] = ((ferc_df.plant_name.str.match('Pawnee')) &
                        (ferc_df.utility_id==145))

###########################################################################
# Mayden 1 & 2 -- operated by PSCO, partly owned by PacifiCorp
ferc_masks['hayden_psco'] = ((ferc_df.plant_name.str.match('Hayden')) &
                             (ferc_df.utility_id==145))
ferc_masks['hayden_pc'] = ((ferc_df.plant_name.str.match('Hayden')) &
                           (ferc_df.utility_id==134))

###########################################################################
# Craig station is a big ownership mess...
ferc_masks['craig_psco'] = ((ferc_df.plant_name.str.match('Craig')) &
                            (ferc_df.utility_id==145))
ferc_masks['craig_pc'] = ((ferc_df.plant_name.str.match('Craig')) &
                          (ferc_df.utility_id==134))

###########################################################################
# Colstrip is complicated. No EIA data for some reason -- cost are all
# unreportd, but there should be other data, no? But it's all NA. I have
# included what I think the masks should be to capture those units anyway,
# in case we can fix it later or there's some real data there monthly...
# FERC ownership adds up to 1400MW in 2008 and later, but less earlier.
# Did the plant used to be largely owned by a public power entity?
ferc_masks['colstrip_nwc'] = ((ferc_df.plant_name.str.match('.*Colstrip.*')) &
                              (ferc_df.utility_id==122)) # 240 MW 2004-2016
ferc_masks['colstrip_pc'] = ((ferc_df.plant_name.str.match('.*Colstrip.*')) &
                             (ferc_df.utility_id==134)) # 155 MW 2004-2016
ferc_masks['colstrip_avista'] = ((ferc_df.plant_name.str.match('.*Colstrip.*')) &
                                 (ferc_df.utility_id==187)) # 233 MW 2004-2016
ferc_masks['colstrip_pse'] = ((ferc_df.plant_name.str.match('.*Colstrip.*')) &
                              (ferc_df.utility_id==150)) # 377 + 433 MW 2008-2016

###########################################################################
# Hunter plant. No EIA data coming through on 2014-2015 at annual freq.
# PacifiCorp is reporting the three units independently for some reason,
# which is great.
ferc_masks['hunter_1'] = ((ferc_df.plant_name.str.match('^Hunter.*1$')) &
                          (ferc_df.utility_id==134))
ferc_masks['hunter_2'] = ((ferc_df.plant_name.str.match('^Hunter.*2$')) &
                          (ferc_df.utility_id==134))
ferc_masks['hunter_3'] = ((ferc_df.plant_name.str.match('^Hunter.*3$')) &
                          (ferc_df.utility_id==134))

###########################################################################
# Jim Bridger -- 4 units, jointly owned by PacifiCorp & Idaho Power Co.
ferc_masks['jim_bridger_pc'] = ((ferc_df.plant_name.str.match('^Jim Bridger$')) &
                                (ferc_df.utility_id==134))
ferc_masks['jim_bridger_idpc'] = ((ferc_df.plant_name.str.match('^Jim Bridger$')) &
                                  (ferc_df.utility_id==70))

ferc_masks['huntington'] = ((ferc_df.plant_name.str.match('^Huntington$')) &
                            (ferc_df.utility_id==134))

###########################################################################
# Maramec - 4 units, one owner
ferc_masks['maramec'] = ((ferc_df.plant_name.str.match('^Meramec$')) &
                            (ferc_df.utility_id==177))

###########################################################################
# North Valmy - 2 units, jointly owned by Sierra Pacific & Idaho Power
ferc_masks['north_valmy_sp'] = ((ferc_df.plant_name.str.match('^Valmy$')) &
                            (ferc_df.utility_id==157))
ferc_masks['north_valmy_ip'] = ((ferc_df.plant_name.str.match('^Valmy$')) &
                            (ferc_df.utility_id==70))


###########################################################################
# Big Stone, three FERC respondents
ferc_masks['big_stone_ot'] = ((ferc_df.plant_id_pudl==55) &
                                (ferc_df.utility_id==132))
ferc_masks['big_stone_nwc'] = ((ferc_df.plant_id_pudl==55) &
                                (ferc_df.utility_id==122))
ferc_masks['big_stone_mdu'] = ((ferc_df.plant_id_pudl==55) &
                                (ferc_df.utility_id==95))

###########################################################################
# Four Corners... 5 units. 5 owners. APS reports units after 2006
ferc_masks['four_courners_elp'] = (ferc_df.plant_name.str.match('Four Corners')&
                                     (ferc_df.utility_id==49))
ferc_masks['four_courners_tep'] = (ferc_df.plant_name.str.match('Four Corners')&
                                     (ferc_df.utility_id==176))
ferc_masks['four_courners_sce'] = (ferc_df.plant_name.str.match('Four Corners')&
                                     (ferc_df.utility_id==161))

ferc_masks['four_courners_1_aps'] = (ferc_df.plant_name.str.match('Four Corners 1')
                                     |(ferc_df.plant_name=='Four Corners') &
                                     (ferc_df.utility_id==7))
ferc_masks['four_courners_2_aps'] = (ferc_df.plant_name.str.match('Four Corners 2')
                                     |(ferc_df.plant_name=='Four Corners') &
                                     (ferc_df.utility_id==7))
ferc_masks['four_courners_3_aps'] = (ferc_df.plant_name.str.match('Four Corners 3')
                                     |(ferc_df.plant_name=='Four Corners') &
                                     (ferc_df.utility_id==7))
ferc_masks['four_courners_4_aps'] = (ferc_df.plant_name.str.match('Four Corners 4')
                                     |(ferc_df.plant_name=='Four Corners') &
                                     (ferc_df.utility_id==7))
ferc_masks['four_courners_5_aps'] = (ferc_df.plant_name.str.match('Four Corners 5')
                                     |(ferc_df.plant_name=='Four Corners') &
                                     (ferc_df.utility_id==7))

ferc_masks['four_courners_4_4c'] = (ferc_df.plant_name.str.match('Four Corners4')&
                                     (ferc_df.utility_id==147))
ferc_masks['four_courners_5_4c'] = (ferc_df.plant_name.str.match('Four Corners 5')&
                                     (ferc_df.utility_id==147))


ferc_masks['four_courners_1_psnm'] = (ferc_df.plant_name.str.match('Four Corners (1)')&
                                     (ferc_df.utility_id==513))



NameError: name 'ferc_df' is not defined